# Running a fixed-length TPS simulation

This is file runs the main calculation for the fixed length TPS simulation. It requires the file `alanine_dipeptide_fixed_tps_traj.nc`, which is written in the notebook `alanine_dipeptide_fixed_tps_traj.ipynb`.

In this file, you will learn:
* how to set up and run a fixed length TPS simulation

NB: This is a long calculation. In practice, it would be best to export the Python from this notebook, remove the `live_visualizer`, and run non-interactively on a computing node.

In [1]:
import openpathsampling as paths

## Load engine, trajectory, and states from file

In [2]:
old_storage = paths.Storage("tps_nc_files/alanine_dipeptide_fixed_tps_traj.nc", "r")

In [3]:
engine = old_storage.engines[0]
C_7eq = old_storage.volumes.find('C_7eq')
alpha_R = old_storage.volumes.find('alpha_R')
traj = old_storage.trajectories[0]
phi = old_storage.cvs.find('phi')
psi = old_storage.cvs.find('psi')
template = old_storage.snapshots[0]

In [4]:
print engine.name
print engine.snapshot_timestep

300K
0.02 ps


## TPS

The only difference between this and the flexible path length example in `alanine_dipeptide_tps_run.ipynb` is that we used a `FixedLengthTPSNetwork`. We selected the `length=400` (8 ps) as a maximum length based on the results from a flexible path length run.

In [5]:
network = paths.FixedLengthTPSNetwork(C_7eq, alpha_R, length=400)

In [6]:
scheme = paths.OneWayShootingMoveScheme(network, selector=paths.UniformSelector(), engine=engine)

In [7]:
initial_conditions = scheme.initial_conditions_from_trajectories(traj)

No missing ensembles.
No extra ensembles.


In [8]:
sampler = paths.PathSampling(storage=paths.Storage("tps_nc_files/alanine_dipeptide_fixed_tps.nc", "w", template),
                             move_scheme=scheme,
                             sample_set=initial_conditions)
#sampler.live_visualizer = paths.StepVisualizer2D(network, phi, psi, [-3.14, 3.14], [-3.14, 3.14])

In [9]:
#sampler.live_visualizer = None

In [10]:
sampler.run(10000)

Working on Monte Carlo cycle number 10000
Running for 124921 seconds -  0.08 steps per second
Expected time to finish: 12 seconds
DONE! Completed 10000 Monte Carlo cycles.


With this done, you can go on to do the fixed-length parts of the analysis in `alanine_dipeptide_tps_analysis.ipynb`.